In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [3]:
train = pd.read_csv(r"C:\Users\Gusti Jogish\Downloads\store_sales\train.csv")
test  = pd.read_csv(r"C:\Users\Gusti Jogish\Downloads\store_sales\test.csv")
stores = pd.read_csv(r"C:\Users\Gusti Jogish\Downloads\store_sales\stores.csv")
holidays = pd.read_csv(r"C:\Users\Gusti Jogish\Downloads\store_sales\holidays_events.csv")
oil = pd.read_csv(r"C:\Users\Gusti Jogish\Downloads\store_sales\oil.csv")

In [4]:
train = train.merge(stores, on="store_nbr", how="left")
test = test.merge(stores, on="store_nbr", how="left")

# Convert date
train["date"] = pd.to_datetime(train["date"])
test["date"]  = pd.to_datetime(test["date"])

In [5]:
lags = [1,3,7,30]
rollings = [7,30]

for lag in lags:
    train[f'lag_{lag}'] = train.groupby(['store_nbr','family'])['sales'].shift(lag)
    test[f'lag_{lag}'] = np.nan  # test belum ada sales

for window in rollings:
    train[f'rolling_{window}'] = train.groupby(['store_nbr','family'])['sales'].shift(1).rolling(window).mean()
    test[f'rolling_{window}'] = np.nan

In [6]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [7]:
for col in ["store_nbr","family","city","state","type","cluster"]:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

In [8]:
features = [col for col in train.columns if col not in ["date","sales"]]
X = train[features]
y = np.log1p(train["sales"])  # log-transform
X_test = test[features]


In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05, shuffle=False)

In [10]:
class TabularDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32) if y is not None else None
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

train_loader = DataLoader(TabularDataset(X_train, y_train), batch_size=1024, shuffle=True)
val_loader   = DataLoader(TabularDataset(X_val, y_val), batch_size=1024)
test_loader  = DataLoader(TabularDataset(X_test), batch_size=1024)

In [11]:
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,1)
        )
    def forward(self, x):
        return self.layers(x).squeeze(1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FeedForwardNN(X_train.shape[1]).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
n_epochs = 50
for epoch in range(n_epochs):
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            val_loss += criterion(preds, yb).item() * xb.size(0)

    print(f"Epoch {epoch+1}/{n_epochs} | Train Loss: {train_loss/len(X_train):.5f} | Val Loss: {val_loss/len(X_val):.5f}")

Epoch 1/50 | Train Loss: 1556541.04547 | Val Loss: 468.06081
Epoch 2/50 | Train Loss: 397873.55819 | Val Loss: 66.42154
Epoch 3/50 | Train Loss: 190709.10977 | Val Loss: 6669080.20909
Epoch 4/50 | Train Loss: 96657.59367 | Val Loss: 15.05472
Epoch 5/50 | Train Loss: 50556.81548 | Val Loss: 11.63578
Epoch 6/50 | Train Loss: 30771.45643 | Val Loss: 27.97355
Epoch 7/50 | Train Loss: 14164.54569 | Val Loss: 94.02078
Epoch 8/50 | Train Loss: 51478.19168 | Val Loss: 6.03899
Epoch 9/50 | Train Loss: 5.58130 | Val Loss: 9.98844
Epoch 10/50 | Train Loss: 743.09511 | Val Loss: 11.92702
Epoch 11/50 | Train Loss: 655.49845 | Val Loss: 104.70447
Epoch 12/50 | Train Loss: 236.23965 | Val Loss: 3.28010
Epoch 13/50 | Train Loss: 54.86712 | Val Loss: 15.77766
Epoch 14/50 | Train Loss: 137.06004 | Val Loss: 16.45038
Epoch 15/50 | Train Loss: 79.31002 | Val Loss: 12.01695
Epoch 16/50 | Train Loss: 8.53963 | Val Loss: 7.78012
Epoch 17/50 | Train Loss: 7.34472 | Val Loss: 6.94812
Epoch 18/50 | Train Loss: 

In [13]:
model.eval()
all_preds = []
with torch.no_grad():
    for xb in test_loader:
        xb = xb.to(device)
        preds = model(xb)
        all_preds.append(preds.cpu())
y_pred_test = np.expm1(torch.cat(all_preds).numpy())

submission = pd.DataFrame({"id": test["id"], "sales": y_pred_test})
submission.to_csv("submission_torch.csv", index=False)
print("✅ Submission ready: submission_torch.csv")

✅ Submission ready: submission_torch.csv
